# Information Extraction

1. Load and split
2. Vectorsearch (retrieval)

TODO:
* paper for extraction

In [1]:
%run loader.ipynb

In [2]:
html_header_splits = load_and_split(**TEST_URLS["goldman"])
[h.metadata for h in html_header_splits]

Fetching pages: 100%|##############################################################################################| 1/1 [00:00<00:00,  3.36it/s]


131987


[{},
 {'Header 2': 'Executive summary'},
 {'Header 2': 'Introduction'},
 {'Header 2': 'AI and geopolitics'},
 {'Header 2': 'The AI incumbents: The US & China'},
 {'Header 2': 'The AI capabilities of geopolitical swing states'},
 {'Header 2': 'The European Union'},
 {'Header 2': 'The United Kingdom'},
 {'Header 2': 'The United Arab Emirates'},
 {'Header 2': 'Israel'},
 {'Header 2': 'Japan, The Netherlands, South Korea, and Taiwan'},
 {'Header 2': 'India'},
 {'Header 2': 'The emerging AI powers'},
 {'Header 2': 'The geopolitics of AI governance'},
 {'Header 2': 'The emerging AI powers'},
 {'Header 2': 'How technological development drives geopolitics'},
 {'Header 2': 'The key debate: scale up or scale down?'},
 {'Header 2': 'The future of AI governance'},
 {'Header 2': 'Charting the chip wars'},
 {'Header 2': 'AI and commerce'},
 {'Header 2': 'Energy'},
 {'Header 2': 'Compute'},
 {'Header 2': 'Data'},
 {'Header 2': 'Models'},
 {'Header 2': 'What to watch'}]

In [6]:
from langchain.document_loaders import OnlinePDFLoader, UnstructuredPDFLoader

loader = UnstructuredPDFLoader(
    "data_pdfs/tepco-tfcd-2023.pdf",
    # mode="elements",
)
data = loader.load()

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 20,
    length_function = len,
    is_separator_regex = False,
)

html_header_splits = text_splitter.split_documents(data)
len(html_header_splits)

12

In [16]:
# https://github.com/nlmatics/llmsherpa?tab=readme-ov-file#read-a-pdf-file
from llmsherpa.readers import LayoutPDFReader
from langchain.docstore.document import Document

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "data_pdfs/tepco-tfcd-2023.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)
html_header_splits = [Document(page_content=chunk.to_context_text()) for chunk in doc.chunks()]

In [17]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import SKLearnVectorStore

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embeddings = HuggingFaceEmbeddings(model_name="WhereIsAI/UAE-Large-V1")

encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5", encode_kwargs=encode_kwargs)

vector_store = SKLearnVectorStore.from_documents(html_header_splits, embeddings)

In [18]:
keywords = "japan".split(" ")
keywords = "目標 削減 50%".split(" ")

found_docs = vector_store.similarity_search_with_relevance_scores(" ".join(keywords))

for doc in found_docs:
    for keyword in keywords:
        if keyword.lower() in doc[0].page_content.lower():
            print(f"* `{keyword}` included")
    print(doc)
    print()

(Document(page_content='社 会 > カーボンニュートラル 目的の明確化\nお客さまの目的を明確化し、 認識共有を図る', metadata={'id': '47faf61a-df70-45d5-94df-9184d20406cb'}), 0.7452211617219464)

* `目標` included
* `削減` included
(Document(page_content='カーボン ハーフ\n※3 速報値。電源調達の工夫等に加え、法規制対応において、非化石証書 の調達量が増加した影響により約53%削減(目標達成) 社 会\n系統利用の最適化 P26\n広域化', metadata={'id': '1fdd1e02-9d28-44a6-84e7-f942dfb6811f'}), 0.743707297601663)

(Document(page_content='社 会 > カーボンニュートラル 目的の明確化\n［ サービス例 ］関連KPI', metadata={'id': '6189fa96-0b2c-4d6e-8911-c1e1f51409d5'}), 0.7404072927691021)

(Document(page_content='今後も選ばれ続ける 教育旅行先 > 社会的価値\n自然環境の保全', metadata={'id': '91503eaf-c11c-4c3a-b468-c370971813c6'}), 0.7400547509937968)



In [19]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama

llm = Ollama(model="llama2:7b-chat")

In [24]:
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

check_template = """<s>[INST]Please check if the following paragraph is related to Japan and rate this relevance to whether low, middle, or high.

{docs}

Helpful Answer:"""

check_template = """<s>[INST]Please extract the sentences exactly regardng Japan.

{docs}

Helpful Answer:"""

check_prompt = PromptTemplate.from_template(check_template)
check_chain = LLMChain(llm=llm, prompt=check_prompt)

In [25]:
for doc in found_docs[:1]:
    print(doc)
    print("---")
    print(check_chain.run(doc))
    print("====")

(Document(page_content='South Korea, Japan, and Taiwan are home to some of the world’s most important semiconductor design and manufacturing companies, as well as semiconductor manufacturing equipment makers. They are also located in critical geographies for global supply chains along the South China Sea and East China Sea. The world’s great powers are dependent on these countries for their own technological competitiveness.  \nThe importance of these countries in multilateral cooperation has been clear for some time. In addition to their roles in the enforcement of export controls, in March 2022, US President Joe Biden reportedly proposed a “CHIP 4” grouping of the US and these three East Asian governments, a move seen as not only aimed at isolating Beijing, but also at creating greater supply-chain diversification and protecting companies’ intellectual property. However, the initiative has not yet made significant progress.  \nAs artificial intelligence becomes a greater part of dail